# Train neural net for actuation matrix

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import sys
import gc
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import wandb
import copy


cwd = os.getcwd() 
parent_dir = os.path.dirname(cwd)
base_dir = os.path.dirname(parent_dir)

src_dir = base_dir + "/src"

sys.path.insert(0, src_dir)
from calibration import ActuationNet, DirectNet, PotentialNet

In [ ]:
#####################################
########### Which eMNS? #############
#####################################
emns = "octomag" # "octomag" or "navion"

if emns.lower() == "octomag":
    data_dir = base_dir + "/data/octomag_data/split_dataset"
    params_dir = cwd + "/params"
elif emns.lower() == "navion":
    data_dir = base_dir + "/data/navion_data/split_dataset"
    params_dir = cwd + "/params_navion"
else:
    raise ValueError("eMNS type not recognized. Please use 'octomag' or 'navion'.")

## Load data

In [ ]:
# Define the data percentage
data_percentage = 5

In [ ]:
training_data_path = data_dir + "/training_data" + (".pkl" if data_percentage == 100 else "_"+str(data_percentage)+".pkl")
validation_data_path = data_dir + "/validation_data" + (".pkl" if data_percentage == 100 else "_"+str(data_percentage)+".pkl")
test_data_path = data_dir + "/test_data.pkl"

training_data = pd.read_pickle(training_data_path)
validation_data = pd.read_pickle(validation_data_path)
test_data = pd.read_pickle(test_data_path)

# Select only relevant columns
pos_cols = ["x", "y", "z"]
field_cols = ["Bx", "By", "Bz"]
em_cols = [col for col in training_data.columns if col.startswith("em_")]

training_data = training_data[pos_cols + field_cols + em_cols]
validation_data = validation_data[pos_cols + field_cols + em_cols]
test_data = test_data[pos_cols + field_cols + em_cols]

print("Data info:")
print(f"Training data points: {len(training_data)}")
print(f"Validation data points: {len(validation_data)}")
print(f"Test data points: {len(test_data)}")

In [ ]:
# Position distributions: x, y, z
bins = 50  # adjust as needed

fig, axes = plt.subplots(1, 3, figsize=(15, 4), sharey=False)
pos_names = ["x", "y", "z"]
datasets = [training_data, validation_data, test_data]
labels = ["train", "val", "test"]
colors = ["C0", "C1", "C2"]

for i, col in enumerate(pos_names):
    ax = axes[i]
    for ds, lab, colr in zip(datasets, labels, colors):
        ax.hist(ds[col], bins=bins, alpha=0.4, label=lab, density=True, color=colr)
    ax.set_title(f"{col} distribution")
    ax.set_xlabel(col)
    ax.set_ylabel("Density")
    if i == 0:
        ax.legend()

fig.suptitle("Position distributions (x, y, z) for train/val/test", y=1.02)
plt.tight_layout()
plt.show()

# Field component distributions: Bx, By, Bz
fig, axes = plt.subplots(1, 3, figsize=(15, 4), sharey=False)
field_names = ["Bx", "By", "Bz"]

for i, col in enumerate(field_names):
    ax = axes[i]
    for ds, lab, colr in zip(datasets, labels, colors):
        ax.hist(ds[col], bins=bins, alpha=0.4, label=lab, density=True, color=colr)
    ax.set_title(f"{col} distribution")
    ax.set_xlabel(col)
    ax.set_ylabel("Density")
    if i == 0:
        ax.legend()

fig.suptitle("Field component distributions (Bx, By, Bz) for train/val/test", y=1.02)
plt.tight_layout()
plt.show()

# Field norm distribution: |B|
# compute norms
train_B_norm = np.linalg.norm(training_data[field_cols].to_numpy(), axis=1)
val_B_norm   = np.linalg.norm(validation_data[field_cols].to_numpy(), axis=1)
test_B_norm  = np.linalg.norm(test_data[field_cols].to_numpy(), axis=1)

fig, ax = plt.subplots(figsize=(6, 4))

ax.hist(train_B_norm, bins=bins, alpha=0.4, density=True, label="train", color="C0")
ax.hist(val_B_norm,   bins=bins, alpha=0.4, density=True, label="val",   color="C1")
ax.hist(test_B_norm,  bins=bins, alpha=0.4, density=True, label="test",  color="C2")

ax.set_title("|B| distribution for train/val/test")
ax.set_xlabel("|B|")
ax.set_ylabel("Density")
ax.legend()

plt.tight_layout()
plt.show()

## Setup normalization

In [ ]:
normalize_pos = False # We did not normalize position
normalize_field = True

normalization_params = {"input": None, "output": None}

if normalize_pos:
    pos_mean = training_data[pos_cols].mean().to_numpy()  # shape (3,)
    pos_std  = training_data[pos_cols].std().to_numpy()
    normalization_params["input"] = {
        "mean": pos_mean,
        "std": pos_std,
    }

if normalize_field:
    field_mean = training_data[field_cols].mean().to_numpy()  # shape (3,)
    field_std  = training_data[field_cols].std().to_numpy()
    normalization_params["output"] = {
        "mean": field_mean,
        "std": field_std,
    }

if normalize_pos:
    print("Position normalization parameters:")
    print(f"Mean: {normalization_params['input']['mean']}")
    print(f"Std: {normalization_params['input']['std']}")
else:
    print("Position normalization not applied.")

if normalize_field:
    print("Field normalization parameters:")
    print(f"Mean: {normalization_params['output']['mean']}")
    print(f"Std: {normalization_params['output']['std']}")

## Build Torch Datasets

In [ ]:
# Build a torch Dataset class

class MagneticDataset(Dataset):
    def __init__(self, pos, currents, field):
        """
        pos:      np.ndarray (N, 3)
        currents: np.ndarray (N, 8)
        field:    np.ndarray (N, 3)
        """

        self.pos = torch.from_numpy(pos.astype(np.float32))
        self.curr = torch.from_numpy(currents.astype(np.float32))
        self.field = torch.from_numpy(field.astype(np.float32))


    def __len__(self):
        return self.pos.shape[0]

    def __getitem__(self, idx):
        return self.pos[idx], self.curr[idx], self.field[idx]

In [ ]:
# Extract data to numpy arrays
pos_train   = training_data[pos_cols].to_numpy()      # (N_train, 3)
field_train = training_data[field_cols].to_numpy()    # (N_train, 3)
curr_train  = training_data[em_cols].to_numpy()       # (N_train, 8)

pos_val   = validation_data[pos_cols].to_numpy()
field_val = validation_data[field_cols].to_numpy()
curr_val  = validation_data[em_cols].to_numpy()

pos_test   = test_data[pos_cols].to_numpy()
field_test = test_data[field_cols].to_numpy()
curr_test  = test_data[em_cols].to_numpy()


# Build datasets
train_dataset = MagneticDataset(
    pos_train,
    curr_train,
    field_train
)

val_dataset = MagneticDataset(
    pos_val,
    curr_val,
    field_val
)

test_dataset = MagneticDataset(
    pos_test,
    curr_test,
    field_test
)

## Training 🌊🏄‍♂️🤙

In [ ]:
# Define the model
model_class = PotentialNet
hidden_dims = (512, 512, 512)

In [ ]:
data_percentage_name = str(data_percentage)
structure_name = "x".join([str(dim) for dim in hidden_dims])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model_class(
    name=model_class.__name__ + "_" + data_percentage_name + "_" + structure_name,
    hidden_dims=hidden_dims,
    num_coils=8 if emns.lower() == "octomag" else 3,
    position_normalization_dict=normalization_params["input"],
    field_normalization_dict=normalization_params["output"],
).to(device)

print("Model name is: " + model.name)


batch_size = 512
num_epochs = 10000
adam_lr = 1e-3

early_stopping_patience = 10
early_stopping_threshold = 0.001

# Adapt patience based on reduced dataset
# Load full dataset
adjusted_patience = max(1, int(early_stopping_patience / data_percentage * 100))
early_stopping_patience = adjusted_patience
print(f"Adjusted early stopping patience: {early_stopping_patience}")

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=adam_lr)

In [ ]:
# Dataset to DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False)

# Init wandb
wandb.init(
    project="ReproducibleMagneticNetworks",
    name=model.name,
    config={
        "batch_size": batch_size,
        "num_epochs": num_epochs,
        "lr": adam_lr,
        "model": model.name,
        "normalize_pos": normalize_pos,
        "normalize_field": normalize_field,
    },
)

wandb.watch(model, criterion, log="all", log_freq=100)

# --- early stopping state ---
baseline_rmse = None          # first RMSE of current run
plateau_counter = 0

# --- best model tracking (for saving) ---
best_val_rmse = float('inf')
best_state_dict = copy.deepcopy(model.state_dict())
best_epoch = 0

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0

    # accumulators for norms (train)
    J_frob_train_sum = 0.0
    b_norm_train_sum = 0.0
    train_batches = 0
    has_b_train = False

    # accumulator for train SE
    se_sum_train = 0.0

    for pos_batch, curr_batch, field_batch in train_loader:
        pos_batch   = pos_batch.to(device)
        curr_batch  = curr_batch.to(device)
        field_batch = field_batch.to(device)

        optimizer.zero_grad()
        out = model(pos_batch, curr_batch)

        # Handle (field, J) or (field, J, b)
        J_batch = None
        b_batch = None

        if isinstance(out, (list, tuple)):
            if len(out) == 3:
                field_pred, J_batch, b_batch = out
            elif len(out) == 2:
                field_pred, J_batch = out
            else:
                field_pred = out[0]
        else:
            field_pred = out

        loss = criterion(field_pred, field_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * pos_batch.size(0)
        train_batches += 1

        # ---- Frobenius norm of J and L2 norm of b over batch ----
        if J_batch is not None:
            # J_batch: (B, 3, 8) -> per-sample Frobenius, then mean over B
            J_frob_batch = J_batch.view(J_batch.size(0), -1).norm(dim=1).mean().item()
            J_frob_train_sum += J_frob_batch

        if b_batch is not None:
            # b_batch: (B, 3) -> per-sample L2 norm, then mean over B
            b_norm_batch = b_batch.norm(dim=1).mean().item()
            b_norm_train_sum += b_norm_batch
            has_b_train = True

        pred_np_train = field_pred.detach().cpu().numpy()  # (B, 3)
        true_np_train = field_batch.detach().cpu().numpy() # (B, 3)

        se_sum_train += np.sum((pred_np_train - true_np_train) ** 2)

    train_loss /= len(train_dataset)
    J_frob_train = J_frob_train_sum / train_batches if train_batches > 0 else 0.0
    b_norm_train = b_norm_train_sum / train_batches if (train_batches > 0 and has_b_train) else 0.0

    # train RMSE
    n_vals_train = len(train_dataset)
    mse_train  = se_sum_train / n_vals_train
    rmse_train = np.sqrt(mse_train)

    # ----- validation -----
    model.eval()
    val_loss = 0.0 
    se_sum = 0.0

    # accumulators for norms (val)
    J_frob_val_sum = 0.0
    b_norm_val_sum = 0.0
    val_batches = 0
    has_b_val = False

    with torch.no_grad():
        for pos_batch, curr_batch, field_batch in val_loader:
            pos_batch   = pos_batch.to(device)
            curr_batch  = curr_batch.to(device)
            field_batch = field_batch.to(device)

            out = model(pos_batch, curr_batch)
            J_batch = None
            b_batch = None

            if isinstance(out, (list, tuple)):
                if len(out) == 3:
                    field_pred, J_batch, b_batch = out
                elif len(out) == 2:
                    field_pred, J_batch = out
                else:
                    field_pred = out[0]
            else:
                field_pred = out

            loss = criterion(field_pred, field_batch)
            val_loss += loss.item() * pos_batch.size(0)
            val_batches += 1

            # norms
            if J_batch is not None:
                J_frob_batch = J_batch.view(J_batch.size(0), -1).norm(dim=1).mean().item()
                J_frob_val_sum += J_frob_batch
            if b_batch is not None:
                b_norm_batch = b_batch.norm(dim=1).mean().item()
                b_norm_val_sum += b_norm_batch
                has_b_val = True

            pred_np  = field_pred.cpu().numpy()    # (B, 3)
            true_np  = field_batch.cpu().numpy()   # (B, 3)

            se_sum += np.sum((pred_np - true_np) ** 2)

    val_loss /= len(val_dataset)
    J_frob_val = J_frob_val_sum / val_batches if val_batches > 0 else 0.0
    b_norm_val = b_norm_val_sum / val_batches if (val_batches > 0 and has_b_val) else 0.0

    # MSE / RMSE
    n_vals = len(val_dataset)  # N samples
    mse  = se_sum / n_vals
    rmse = np.sqrt(mse)

    # -------- best model tracking --------
    if rmse < best_val_rmse:
        best_val_rmse = rmse
        best_state_dict = copy.deepcopy(model.state_dict())
        best_epoch = epoch + 1

    # -------- early stopping vs first --------
    if baseline_rmse is None:
        # first epoch initializes the baseline
        baseline_rmse = rmse
        plateau_counter = 0
    else:
        improvement = baseline_rmse - rmse
        if improvement > early_stopping_threshold:
            # improved by more than threshold vs baseline -> reset run
            baseline_rmse = rmse
            plateau_counter = 0
        else:
            # no significant improvement vs baseline
            plateau_counter += 1

    if plateau_counter >= early_stopping_patience:
        print(
            f"Early stopping at epoch {epoch+1} "
            f"due to no > {early_stopping_threshold:.3g} improvement in val RMSE "
            f"for {early_stopping_patience} epochs."
        )
        break

    # log to wandb
    log_dict = {
        "epoch": epoch + 1,
        "train_loss_norm": train_loss,
        "val_loss_norm": val_loss,
        "train_RMSE": rmse_train,
        "val_RMSE": rmse,
        "J_frob_train": J_frob_train,
        "J_frob_val": J_frob_val,
    }
    if has_b_train or has_b_val:
        log_dict["b_norm_train"] = b_norm_train
        log_dict["b_norm_val"] = b_norm_val

    wandb.log(log_dict)

    print(
        f"Epoch {epoch+1:03d} | "
        f"train_loss (norm) = {train_loss:.6f} | "
        f"val_loss (norm) = {val_loss:.6f} | "
        f"train_RMSE = {rmse_train:.3e} | "
        f"val_RMSE = {rmse:.3e}"
    )

# ---- Restore best weights and save best model ----
model.load_state_dict(best_state_dict)

model_save_path = model.save(folder=params_dir)
print(f"Best model (epoch {best_epoch}, val_RMSE={best_val_rmse:.3e}) saved to: {model_save_path}")

## Check how it performs on validation

In [ ]:
model.eval()

validation_data = test_data.copy()
pos_val  = validation_data[["x", "y", "z"]].to_numpy().astype(np.float32)
curr_val = validation_data[em_cols].to_numpy().astype(np.float32)

pos_tensor  = torch.from_numpy(pos_val).to(device)   # (N, 3)
curr_tensor = torch.from_numpy(curr_val).to(device)  # (N, 8)

with torch.no_grad():
    out = model(pos_tensor, curr_tensor)

    if isinstance(out, (list, tuple)):
        field_pred = out[0]
    else:
        field_pred = out

    pred_np = field_pred.cpu().numpy()


validation_data["Bx_pred"] = pred_np[:, 0]
validation_data["By_pred"] = pred_np[:, 1]
validation_data["Bz_pred"] = pred_np[:, 2]

validation_data["B_mag"] = np.sqrt(
    validation_data["Bx"]**2 +
    validation_data["By"]**2 +
    validation_data["Bz"]**2
)
validation_data["B_mag_pred"] = np.sqrt(
    validation_data["Bx_pred"]**2 +
    validation_data["By_pred"]**2 +
    validation_data["Bz_pred"]**2
)

# Compute errors
validation_data["Bx_error"] = validation_data["Bx_pred"] - validation_data["Bx"]
validation_data["By_error"] = validation_data["By_pred"] - validation_data["By"]
validation_data["Bz_error"] = validation_data["Bz_pred"] - validation_data["Bz"]

validation_data["B_mag_error"] = validation_data["B_mag_pred"] - validation_data["B_mag"]
validation_data["B_mag_error_rel"] = validation_data["B_mag_error"] / validation_data["B_mag"].replace(0, np.nan)

validation_data["B_angle_error"] = np.arccos(
    (validation_data["Bx"] * validation_data["Bx_pred"] +
     validation_data["By"] * validation_data["By_pred"] +
     validation_data["Bz"] * validation_data["Bz_pred"]) /
    (validation_data["B_mag"] * validation_data["B_mag_pred"]).replace(0, np.nan)
) * (180.0 / np.pi)  # in degrees

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))


rel_err = validation_data["B_mag_error_rel"].to_numpy()
abs_rel_err = np.abs(rel_err)

# avoid log(0): clip zeros to a tiny positive value
eps = 1e-8
abs_rel_err = np.clip(abs_rel_err, eps, None)

# log-spaced bin edges, e.g. from ~min to ~max
bin_min = max(abs_rel_err.min(), 1e-6)
bin_max = abs_rel_err.max()
bins = np.logspace(np.log10(bin_min), np.log10(bin_max), 40)  # 40 log bins

axes[0].hist(
    abs_rel_err,
    bins=bins,
    alpha=0.7,
    color="blue",
    density=False,
)
axes[0].set_xscale("log")  # so bins like [1e-3, 1e-2], [1e-2, 1e-1], ...
axes[0].set_title("Histogram of |B| Relative Error")
axes[0].set_xlabel("|B_mag_relative_error|")
axes[0].set_ylabel("Count")
axes[0].grid(False)


axes[1].hist(
    validation_data["B_angle_error"],
    bins=100,
    alpha=0.7,
    color="blue",
    density=False,
)
axes[1].set_title("Histogram of B Angle Error")
axes[1].set_xlabel("B_angle Error (degrees)")
axes[1].set_ylabel("Count")
axes[1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Compute |B_true| and distance from center r
B_true = validation_data[["Bx", "By", "Bz"]].to_numpy()
pos    = validation_data[["x", "y", "z"]].to_numpy()

B_mag  = np.linalg.norm(B_true, axis=1)   # (N,)
r      = np.linalg.norm(pos, axis=1)      # (N,)

# errors
err_mag_rel = np.abs(validation_data["B_mag_error_rel"].to_numpy())
err_angle   = np.abs(validation_data["B_angle_error"].to_numpy())  # or keep sign if you want

# Filter out too small |B|
B_threshold = 0.0  # mT
mask = B_mag >= B_threshold

B_mag      = B_mag[mask]
r          = r[mask]
err_mag_rel = err_mag_rel[mask]
err_angle   = err_angle[mask]

# Binning
n_bins = 100
n_bins_B = n_bins
n_bins_r = n_bins


B_min, B_max = B_mag.min(), B_mag.max()
r_min, r_max = r.min(), r.max()

B_edges = np.linspace(B_min, B_max, n_bins_B + 1)
r_edges = np.linspace(r_min, r_max, n_bins_r + 1)

# --------- 2D binned mean relative magnitude error ---------
err_sum_mag, _, _ = np.histogram2d(B_mag, r, bins=[B_edges, r_edges], weights=err_mag_rel)
count_mag,    _, _ = np.histogram2d(B_mag, r, bins=[B_edges, r_edges])

mean_err_mag = np.zeros_like(err_sum_mag)
mask_mag = count_mag > 0
mean_err_mag[mask_mag] = err_sum_mag[mask_mag] / count_mag[mask_mag]
mean_err_mag[~mask_mag] = np.nan

# --------- 2D binned mean angle error ---------
err_sum_ang, _, _ = np.histogram2d(B_mag, r, bins=[B_edges, r_edges], weights=err_angle)
count_ang,   _, _ = np.histogram2d(B_mag, r, bins=[B_edges, r_edges])

mean_err_ang = np.zeros_like(err_sum_ang)
mask_ang = count_ang > 0
mean_err_ang[mask_ang] = err_sum_ang[mask_ang] / count_ang[mask_ang]
mean_err_ang[~mask_ang] = np.nan

# --------- Plot side by side ---------
fig, axes = plt.subplots(1, 2, figsize=(14, 6), sharex=True, sharey=True)

# Left: relative |B| error
pcm1 = axes[0].pcolormesh(B_edges, r_edges, mean_err_mag.T, shading="auto")
cb1 = fig.colorbar(pcm1, ax=axes[0])
cb1.set_label("Mean |B| relative error")
axes[0].set_xlabel("|B_true|")
axes[0].set_ylabel("‖position‖ (m)")
axes[0].set_title("Relative |B| error vs |B| and radius")

# Right: angle error
pcm2 = axes[1].pcolormesh(B_edges, r_edges, mean_err_ang.T, shading="auto")
cb2 = fig.colorbar(pcm2, ax=axes[1])
cb2.set_label("Mean angle error (deg)")
axes[1].set_xlabel("|B_true|")
axes[1].set_title("Angle error vs |B| and radius")

plt.tight_layout()
plt.show()